In [1]:
# Run some setup code for this notebook.
import random
import numpy as np
import os
import utils as ut
import wave
import SoundUtils as SU
import pylab
import Spectograms as spectograms
import forwardPath as forward
import matplotlib 
import matplotlib.image as pltimg
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
import math
from PIL import Image

%load_ext autoreload
%autoreload 2

In [2]:
path_to_flac_test_folder_track = "/Volumes/Storage/TIMIT/timit/train/dr1/fcjf0/SA1.WAV"
temp_folder = "/Volumes/Storage/Temp"
temp_image_to_process = "/Volumes/Storage/processing.jpg"
array_inversion_file = "/Users/Sergey/Thesis/SpectrogramInversion1.02b/invertMe.mat"
array_inversion_shown_file = "/Users/Sergey/Thesis/SpectrogramInversion1.02b/invertMeShown.mat"


temp = "temp.jpg"

In [3]:
wav_file = SU.PCM2Wav(path_to_flac_test_folder_track, temp_folder)

In [4]:
frames, frame_rate = spectograms.get_wav_info(wav_file)

In [5]:
wav_file

'/Volumes/Storage/Temp/SA1.wav'

In [6]:
windowSize = 256
frameStep = 128
waveform = frames
spect = spectograms.CreateSpectogram(waveform)
magSpec, freqs, times = spectograms.SpectrogramForDisplay(spect, frame_rate, frameStep)

In [7]:
scipy.io.savemat(array_inversion_file, mdict={'spect':spect})

In [8]:
A = np.real(magSpec)
normalized, minEl, maxEl = spectograms.normalize(A)
normedRGB = spectograms.toRGB(normalized)

In [9]:
from PIL import Image
img = Image.fromarray(normedRGB)
img.save(temp)

In [20]:
A = pltimg.imread(temp)
grayArr = spectograms.fromRgb(A)
original = np.flipud(spectograms.originalFromNormalized(grayArr, minEl, maxEl))
backwardSpec = spectograms.backwardDisplaySpectogram(spect, original, windowSize)

In [ ]:
scipy.io.savemat(array_inversion_shown_file, mdict={'spect2':backwardSpec})